In [4]:
import json

def get_key():
    with open("secretkey.json", "r") as json_file:
        return json.load(json_file)["api_key"]


In [5]:
# from docx import Document
# def docx_to_txt(docx_path, txt_path):
#     doc = Document(docx_path)
#     with open(txt_path, "w", encoding="utf-8") as txt_file:
#         for para in doc.paragraphs:
#             txt_file.write(para.text + "\n")
# docx_path = "path/to/your/document.docx"
# txt_path = "path/to/your/output.txt"

# docx_to_txt(docx_path, txt_path)
# print("Conversion complete!")

In [6]:
import requests
import json
import re
# working formater
def modelcall(KEY, deploymentName, SYS_prompt, USER_prompt):
    ENDPOINT = f"https://intern2024openai.openai.azure.com/openai/deployments/{deploymentName}/chat/completions?api-version=2024-02-15-preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": KEY,
    }
    # Correct message structure
    message = [
        {"role": "system", "content": SYS_prompt},
        {"role": "user", "content": USER_prompt},
    ]
    payload = {
        "messages": message,
        "temperature": 0,
        "top_p": 0,
    }
    # Send request
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() 
        data = response.json()
    except requests.exceptions.RequestException as e:
        try:
            error_response = response.json()
            if "error" in error_response:
                content_filter_result = (
                    error_response["error"]
                    .get("innererror", {})
                    .get("content_filter_result", {})
                )
                filtered_items = {
                    k: v
                    for k, v in content_filter_result.items()
                    if v["severity"] != "safe"
                }

                if filtered_items:
                    error_message = (
                        "Filtered due to the following reasons:"
                    )
                    for item, details in filtered_items.items():
                        error_message += f" ╽{item.capitalize()}: severity = {details['severity']}, filtered = {details['filtered']}╽"
                    return ["error", "filtered", error_message]
                else:
                    return [
                        "error", 
                        "filtering",
                        "An error occurred, but no content filter issues were found.",
                    ]
            else:
                return [
                    "error",
                    "no_details",
                    "An error occurred, but no detailed error information is available. Goodluck",
                ]
        except json.JSONDecodeError:
            return ["error", "json_decode", f"Request error: {e}"]
        except Exception as e:
            return ["error", "unexpected", f"An unexpected error occurred: {e}"]

    content = data["choices"][0]["message"]["content"]
    pattern = r"```json(.*?)```"
    match = re.search(pattern, content, re.DOTALL)
    if match:
        try:
            return ["success", json.loads(match.group(1).strip())]
        except json.JSONDecodeError as e:
            return ["error", "json_parse", f"JSON parsing error: {e}\n\nContent: {content}"]
    else:
        try:
            return ["success", json.loads(content)]
        except json.JSONDecodeError as e:
            return [
                "error",
                "json_parse",
                f"assistant_response: {content}"
            ]
        except Exception as e:
            return ["error", "unexpected", f"An unexpected error occurred: {e}"]


USER_prompt = input("Hi, What powerpoint content would you like to generate:\n")
SYS_prompt = """Instruct:
You are to take the user question and seperate it into 3 sections labeled as "stem", "lead-in", "options" or 2 section labeled as "lead-in", "options" if "stem" is not present.

The "stem" is the main body of the question, providing the context or scenario. The "lead-in" is the specific question or directive that follows the "stem", indicating what the examinee needs to determine or answer. The "options" are the multiple-choice answers provided for the examinee to select from.

If unable to extract from a question leave "status" as "fail". "status" is only "success" if content of "lead-in" is not blank and "options" is not an empty.

There can be more then one question.

Guideline:
return in json format having the question id, status, data (stem, lead-in, options).

Example:
Question 1:	Which of the following adaptions to cell injury is described as an increased number of cells within an organ?	

	(A)	Atrophy
	(B)	Dysplasia
	(C)	Hyperplasia
	(D)	Hypertrophy
	(E)	Metaplasia
		
Question 2: DNA replication is known to be semi-conservative. Which of the following statements best describes this semi-conservative nature?

Question 3:	A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed. Which of the following mechanisms best explains the lysis of RBCs?

	(A)	RBCs are biconcave in shape
	(B)	RBCs are lysed because Osmo > Osmi
	(C)	RBCs cannot pump Na+ out effectively
	(D)	RBCs do not have a nucleus
	(E)	RBCs take up the hypotonic solution too rapidly through pinocytosis


is returned as

[
    {
        "id": 1,
        "status": "success",
        "data": [{
            "stem": "",
            "lead-in": "Which of the following adaptions to cell injury is described as an increased number of cells within an organ?",
            "options": {
                "A": "Atrophy",
                "B": "Dysplasia",
                "C": "Hyperplasia",
                "D": "Hypertrophy",
                "E": "Metaplasia"
            }
        }]
    },
    {
        "id": 2,
        "status": "fail",
        "data": [{
            "stem": "DNA replication is known to be semi-conservative.",
            "lead-in": "Which of the following statements best describes this semi-conservative nature?",
            "options": {}
            }]
    },
    {
        "id": 3,
        "status": "success",
        "data": [{
            "stem": "A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.",
            "lead-in": "Which of the following mechanisms best explains the lysis of RBCs?",
            "options": {
                "A": "RBCs are biconcave in shape",
                "B": "RBCs are lysed because Osmo > Osmi",
                "C": "RBCs cannot pump Na+ out effectively",
                "D": "RBCs do not have a nucleus",
                "E": "RBCs take up the hypotonic solution too rapidly through pinocytosis"
            }
        }]
    }
]
"""
modelcall(KEY=get_key(), deploymentName = "gpt-35-thread1", SYS_prompt=SYS_prompt, USER_prompt=USER_prompt)

['success',
 [{'id': 1,
   'status': 'success',
   'data': [{'stem': 'Which of the following adaptions to cell injury is described as an increased number of cells within an organ?',
     'lead-in': '',
     'options': {'A': 'Atrophy',
      'B': 'Dysplasia',
      'C': 'Hyperplasia',
      'D': 'Hypertrophy',
      'E': 'Metaplasia'}}]},
  {'id': 2,
   'status': 'success',
   'data': [{'stem': 'A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.',
     'lead-in': 'Which of the following mechanisms best explains the lysis of RBCs?',
     'options': {'A': 'RBCs are biconcave in shape',
      'B': 'RBCs are lysed because Osmo > Osmi',
      'C': 'RBCs cannot pump Na+ out effectively',
      'D': 'RBCs do not have a nucleus',
      'E': 'RBCs take up the hypotonic solution too rapid

In [ ]:
import os
import requests
import base64
import re

# Configuration
KEY = get_key()

user_input = input("Hi, What powerpoint content would you like to generate:\n")
SYS_prompt = """Instruct:
You are to take the user question and seperate it into 3 sections labeled as stem, lead-in, options or 2 section labeled as lead-in, options if stem is not present.

The stem is the main body of the question, providing the context or scenario. The lead-in is the specific question or directive that follows the stem, indicating what the examinee needs to determine or answer. The options are the multiple-choice answers provided for the examinee to select from.

There can be more then one question.

Guideline:
return in json format

Example:
Question 1:	Which of the following adaptions to cell injury is described as an increased number of cells within an organ?	

	(A)	Atrophy
	(B)	Dysplasia
	(C)	Hyperplasia
	(D)	Hypertrophy
	(E)	Metaplasia
		


Question 2:	A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.

Which of the following mechanisms best explains the lysis of RBCs?

	(A)	RBCs are biconcave in shape
	(B)	RBCs are lysed because Osmo > Osmi
	(C)	RBCs cannot pump Na+ out effectively
	(D)	RBCs do not have a nucleus
	(E)	RBCs take up the hypotonic solution too rapidly through pinocytosis


is returned as

[
  {
    "stem": "Which of the following adaptions to cell injury is described as an increased number of cells within an organ?",
    "lead_in": "",
    "options": {
      "A": "Atrophy",
      "B": "Dysplasia",
      "C": "Hyperplasia",
      "D": "Hypertrophy",
      "E": "Metaplasia"
    }
  },
  {
    "stem": "A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.",
    "lead_in": "Which of the following mechanisms best explains the lysis of RBCs?",
    "options": {
      "A": "RBCs are biconcave in shape",
      "B": "RBCs are lysed because Osmo > Osmi",
      "C": "RBCs cannot pump Na+ out effectively",
      "D": "RBCs do not have a nucleus",
      "E": "RBCs take up the hypotonic solution too rapidly through pinocytosis"
    }
  }
]
"""

headers = {
    "Content-Type": "application/json",
    "api-key": KEY,
}

# Payload for the request
payload = {
    "messages": [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": SYS_prompt,
                }
            ],
        },
        {"role": "user", "content": [{"type": "text", "text": user_input}]},
    ],
    "temperature": 0,
    "top_p": 0,
}

deploymentName = "gpt-35-thread1"
ENDPOINT = f"https://intern2024openai.openai.azure.com/openai/deployments/{deploymentName}/chat/completions?api-version=2024-02-15-preview"





{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': "I'm sorry, as an AI language model, I cannot generate the lyrics of copyrighted songs without permission from the owner. It is important to respect intellectual property rights.", 'role': 'assistant'}}], 'created': 1721700419, 'id': 'chatcmpl-9nzDX0pLxWbIgRyoAPBRo8L6qoEAr', 'model': 'gpt-35-turbo', 'object': 'chat.completion', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'system_fingerprint': None, 'usage': {'completion_tokens': 3

JSONDecodeError: Expecting value: line 1 column 1 (char 0)


Question 5:	A nucleosome is a fundamental unit of DNA packaging in eukaryotes, consisting of a segment of DNA wound in sequence around eight histone protein cores. 

Which event involves disassembly of the nucleosome?	
	
	(A)	Chemotherapy drug treatment
	(B)	Mitosis
	(C)	Protein translation
	(D)	RNA transcription
	(E)	Virus infection
		


Question 6:	Following infection of a host cell, the Listeria bacterium can infect neighbouring host cells and evade intracellular detection by generating actin comet tails for propulsion.

Which of the following is the most important nucleator of actin filament growth?
	
	(A)	Arp2/3
	(B)	Myosin
	(C)	Phosphatidylinositol 3-phosphate
	(D)	Profilin
	(E)	γ-tubulin
		

 

Question 7:	DNA replication is known to be semi-conservative.

Which of the following statements best describes this semi-conservative nature?
	
	(A)	After DNA replication, each daughter cell contains one newly synthesised DNA strand and one DNA strand inherited from the parent cell
	(B)	DNA polymerases need an RNA primer to initiate replication
	(C)	DNA replication only occurs during the S-phase of the cell cycle
	(D)	DNA synthesis only occurs in the 5’ to 3’ direction, regardless of strand
	(E)	During DNA replication, the leading strand is continuously synthesised whereas the lagging strand is synthesised in short fragments
		


Question 8:	A 45 year old man presents to the clinic with a history of stage I colon cancer. His father was diagnosed at age 49 with colon cancer and his uncle also had colon cancer diagnosed at age 47. His grandmother had endometrial cancer diagnosed at age 51. 

Which of the following genetic defects is most likely responsible for the colon cancer in this patient?
	
	(A)	A gain of function mutation in APC gene 
	(B)	A loss of function mutation in MSH-2 gene affecting DNA mismatched repair 
	(C)	A loss of function mutation in Retinoblastoma gene (Rb) 
	(D)	A loss of function mutation in TP53 
	(E)	A reciprocal translocation involving chromosomes 8 and 14, t(8;14) 


In [ ]:
import os
import requests
import base64
import re

# Configuration
KEY = get_key()

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, "rb").read()).decode("ascii")
user_input = input("Hi, What powerpoint content would you like to generate:\n")
SYS_prompt = """Instruct:
You are to take the user question and seperate it into 3 sections labeled as stem, lead-in, options or 2 section labeled as lead-in, options if stem is not present.

The stem is the main body of the question, providing the context or scenario. The lead-in is the specific question or directive that follows the stem, indicating what the examinee needs to determine or answer. The options are the multiple-choice answers provided for the examinee to select from.

Guideline:
return in a json format

Example:
Question 2:	A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.

Which of the following mechanisms best explains the lysis of RBCs?

	(A)	RBCs are biconcave in shape
	(B)	RBCs are lysed because Osmo > Osmi
	(C)	RBCs cannot pump Na+ out effectively
	(D)	RBCs do not have a nucleus
	(E)	RBCs take up the hypotonic solution too rapidly through pinocytosis

is returned as

{
  "stem": "A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.",
  "lead_in": "Which of the following mechanisms best explains the lysis of RBCs?",
  "options": {
    "A": "RBCs are biconcave in shape",
    "B": "RBCs are lysed because Osmo > Osmi",
    "C": "RBCs cannot pump Na+ out effectively",
    "D": "RBCs do not have a nucleus",
    "E": "RBCs take up the hypotonic solution too rapidly through pinocytosis"
  }
}

"""

headers = {
    "Content-Type": "application/json",
    "api-key": KEY,
}

# Payload for the request
payload = {
    "messages": [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": SYS_prompt,
                }
            ],
        },
        {"role": "user", 
         "content": [{"type": "text", "text": user_input}]},
    ],
    "temperature": 0.7,
    "top_p": 0.95,
}

ENDPOINT = "https://intern2024openai.openai.azure.com/openai/deployments/gpt4-Omni/chat/completions?api-version=2024-02-15-preview"


# Send request
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")
data = response.json()

def extract_json_text(text):
    pattern = r"```json(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None


print(data["choices"][0]["message"]["content"])

{
  "lead_in": "Which of the following mechanisms best explains the lysis of RBCs?",
  "options": {
    "A": "RBCs are biconcave in shape",
    "B": "RBCs are lysed because Osmo > Osmi",
    "C": "RBCs cannot pump Na+ out effectively",
    "D": "RBCs do not have a nucleus",
    "E": "RBCs take up the hypotonic solution too rapidly through pinocytosis"
  }
}


In [ ]:
import openai


def chat_with_gpt(
    api_key, system_prompt, user_prompt, history=None, use_system_prompt=True
):
    openai.api_key = api_key

    if history is None:
        history = []

    # Prepare the message history
    messages = []
    if use_system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages += history
    messages.append({"role": "user", "content": user_prompt})

    # Call OpenAI API
    response = openai.ChatCompletion.create(model="gpt-4", messages=messages)

    # Get the assistant's reply
    assistant_reply = response["choices"][0]["message"]["content"]

    # Update the history
    new_history = history + [
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": assistant_reply},
    ]

    return [assistant_reply, new_history]


# Example usage:
api_key = "your_openai_api_key"
system_prompt = "You are a helpful assistant."
user_prompt = "Hello, how are you?"
history = None

# Initial call with system prompt
response = chat_with_gpt(api_key, system_prompt, user_prompt, history)
print("Assistant Reply:", response[0])
print("Full History:", response[1])

# Subsequent call without system prompt
new_user_prompt = "What can you do?"
response = chat_with_gpt(
    api_key, system_prompt, new_user_prompt, response[1], use_system_prompt=False
)
print("Assistant Reply:", response[0])
print("Full History:", response[1])